In [ ]:
import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of Korea?",
        }
    ],
)
response


ChatCompletion(id='chatcmpl-C3RlfYj177io9PquItFoUiyH3RCVc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='There are two Koreas:\n\n- South Korea (Republic of Korea): Seoul\n- North Korea (Democratic People’s Republic of Korea): Pyongyang\n\nThere is no single capital for a unified Korea today. If you meant a historical capital (e.g., during the Joseon Dynasty), it was Seoul (also known as Hanyang or Hanseong in history). Do you want more detail on either country?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754937759, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=605, prompt_tokens=13, total_tokens=618, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=512, rejected_prediction_tokens=0), prompt_tokens_detai

In [ ]:
print(response.choices[0].message.content)

There are two Koreas:

- South Korea (Republic of Korea): Seoul
- North Korea (Democratic People’s Republic of Korea): Pyongyang

There is no single capital for a unified Korea today. If you meant a historical capital (e.g., during the Joseon Dynasty), it was Seoul (also known as Hanyang or Hanseong in history). Do you want more detail on either country?